In [171]:
import langchain
from langchain_community.document_loaders import Docx2txtLoader
import regex as re
from langchain_astradb import AstraDBVectorStore
from langchain_core.documents import Document as Langchain_Document
from langchain_huggingface import HuggingFaceEmbeddings
from uuid import uuid4
import langgraph
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [79]:
import re
from docx import Document as Dt

def load_docx(file_path):
    """
    Loads a .docx file and extracts its text content.
    
    Args:
        file_path (str): The path to the .docx file.
        
    Returns:
        str: The full text content of the document.
    """
    doc = Dt(file_path)
    full_text = []
    
    # Extract paragraphs from docx
    for para in doc.paragraphs:
        full_text.append(para.text)
    
    # Join all paragraphs into a single string with newline as separator
    return "\n\n".join(full_text)

def split_document_by_paragraphs(document_content):
    """
    Splits document content into paragraphs based on multiple newline characters.
    
    Args:
        document_content (str): The full document text to be split.
        
    Returns:
        list: A list of paragraphs from the document.
    """
    # Regular expression to split paragraphs using 2 or more consecutive newline characters
    paragraphs = re.split(r'\n{2,}', document_content)
    
    # Remove leading/trailing whitespaces and empty paragraphs
    paragraphs = [para.strip() for para in paragraphs if para.strip()]
    
    return paragraphs

def convert_to_page_content(paragraphs):
    """
    Converts paragraphs to the desired page_content and metadata format.
    
    Args:
        paragraphs (list): List of paragraphs from the document.
        
    Returns:
        list: List of dictionaries containing 'page_content' and 'metadata' for each paragraph.
    """
    documents = []
    
    for idx, paragraph in enumerate(paragraphs):
        doc = Langchain_Document(
            page_content=paragraph,
            metadata={
                "source": f"Paragraph {idx + 1}"
            }
        )
        documents.append(doc)
    
    return documents

# Load the document


In [83]:
docx_file_path = "portfolio_text.docx"  # Replace with the path to your .docx file
document_content = load_docx(docx_file_path)

# Split the document into paragraphs
paragraphs = split_document_by_paragraphs(document_content)

# Convert paragraphs to page_content and metadata
formatted_content = convert_to_page_content(paragraphs)

# Example output: Display each paragraph in the desired format
list_of_document=[document for document in formatted_content]


In [85]:
uuids = [str(uuid4()) for _ in range(len(list_of_document))]

In [99]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "all-MiniLM-L6-v2"
hf = HuggingFaceEmbeddings(
    model_name=model_name
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [102]:
from langchain_astradb import AstraDBVectorStore

vector_store = AstraDBVectorStore(
    collection_name="portfolio",
    embedding=hf,
    api_endpoint="https://ab0b8a51-9354-4c38-9d62-0fbbcb8b90cb-us-east-2.apps.astra.datastax.com",
    token="AstraCS:kQJxeuvgFZJZBYLNZYUaZrOU:2bb633a7989422ac09098e6620a818cefcbd2cf44c3af544f457581a3cf51910", 
)

In [105]:
vector_store.add_documents(documents=list_of_document, ids=uuids)

['ba7c7a25-5dbd-4369-b0a3-009a5d3391c1',
 '13b2dcb1-4eb8-4cdc-93d7-151507677375',
 '598844af-8e85-4ad5-8980-099107d4bf58',
 'f162642c-850c-42ad-aaf6-26149a00781d',
 'cb772668-ab05-4a03-9de6-9929dd5a1329',
 'eb522106-9fd7-4219-be4d-5c954ea842f1',
 '09ca8def-dddc-4965-b179-56017f2da7e0']

In [109]:
retreiver=vector_store.as_retriever()

In [115]:
retreiver.invoke("how he develop something")

[Document(id='13b2dcb1-4eb8-4cdc-93d7-151507677375', metadata={'source': 'Paragraph 2'}, page_content='"I approach projects by first breaking down the problem into its core components. For example, in one chatbot I built, the goal was to automate customer support for an e-commerce platform. I designed the chatbot to understand customer queries using NLP models, and then leveraged LangChain to connect the bot with real-time data sources. I also integrated the bot into existing systems, ensuring it could provide relevant information from databases and even handle payments. I focus on creating solutions that are easy to implement and maintain, always with scalability in mind."'),
 Document(id='09ca8def-dddc-4965-b179-56017f2da7e0', metadata={'source': 'Paragraph 7'}, page_content='Paragraph 7:\n"Ultimately, my work is about creating impact. Every AI project I\'ve worked on has been aimed at solving real-world business problems—whether it’s reducing operational costs, automating repetitive

In [122]:
vector_store.similarity_search_with_score("show your portfolio")
from  langchain.chains import RetrievalQA

In [183]:
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPEN_API_KEY")
model=ChatOpenAI(model="gpt-3.5-turbo")
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
qa = RetrievalQA.from_chain_type(
    chain_type='stuff',
    llm=model, retriever=vector_store.as_retriever(),memory=memory
)

In [186]:
qa.invoke("what was my previos question")["result"]

"I'm sorry, but I don't have access to your previous question. If you can provide me with more information or context, I'll do my best to help you."

{'query': 'what was my previous question',
 'chat_history': [HumanMessage(content='did basit build customize chatbot', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Yes, Abdul Basit has experience building customized chatbots. He has developed chatbots to automate customer support for e-commerce platforms, focusing on understanding customer queries using NLP models, integrating with real-time data sources, databases, and handling payments. His approach involves creating solutions that are tailored to the specific needs of businesses and are designed to be intelligent, adaptable, and scalable.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what was my previos question', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm sorry, but I don't have access to your previous question. How can I assist you today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hello', additional_kwargs={}, response_metadata={})

In [180]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='did basit build customize chatbot', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Yes, Abdul Basit has experience building customized chatbots. He has developed chatbots to automate customer support for e-commerce platforms, focusing on understanding customer queries using NLP models, integrating with real-time data sources, databases, and handling payments. His approach involves creating solutions that are tailored to the specific needs of businesses and are designed to be intelligent, adaptable, and scalable.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what was my previos question', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm sorry, but I don't have access to your previous question. How can I assist you today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hello', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I ass